In [ ]:
import tensorflow as tf
import numpy as np
import random

In [ ]:
words = open('names.txt', 'r').read().splitlines()
chars = sorted(list(set(''.join(words + ['.']))))
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}
stoi

In [ ]:
xs, ys = [], []
block_size = 7

random.shuffle(words)
for w in words:
    # print(w)
    context = [stoi['.']] * block_size
    chs = list(w) + ['.']
    for ch in chs:
        ix = stoi[ch]
        xs.append(context)
        ys.append(ix)
        # print(''.join(itos[i] for i in context), '---->', itos[ix])
        context = context[1:] + [ix]

xs = np.array(xs)
ys = np.array(ys)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(block_size,)),
    tf.keras.layers.Embedding(len(chars), 16),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64, activation=None),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Activation('tanh'),
    tf.keras.layers.Dense(units=64, activation=None),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Activation('tanh'),
    tf.keras.layers.Dense(units=64, activation=None),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Activation('tanh'),
    tf.keras.layers.Dense(units=len(chars), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
y = tf.one_hot(ys, len(chars))
model.fit(xs, y, epochs=10, validation_split=0.1, verbose=1);

In [ ]:
y = tf.one_hot(ys, len(chars))
loss, accuracy = model.evaluate(xs, y, verbose=0)
print(f'{loss=:.04f} {accuracy=:.04f}')

In [ ]:
for i in range(5):
    context = ['.'] * block_size
    while True:
        xi = np.array([[stoi[x] for x in context]])
        p = model.predict(xi, verbose=0)[0]
        i = np.random.choice(len(chars), p=p)
        ch = itos[i]
        context = context[1:] + [ch]
        print(ch, end='')
        if ch == '.':
            break
    print()